In [42]:
#BIBLIOTEKI I OPCJE
import pandas as pd
import re
import random
import math
import uuid
import datetime
import numpy as np

import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

#TABELE
# playerTable = pd.DataFrame(columns=['Name', 'Surname', 'City', 'Category']) #to provide player names
# playerTable = pd.read_csv('examplePlayerTable.csv')
#df = pd.read_csv('csv/examplePlayerTable.csv')
#trickTable = pd.read_csv('trickTableJapan.csv')
#df = df.loc[df.index <= 10]
df = pd.read_csv('csv/masterTable.csv')
#MODUŁY
%run tableFunction.ipynb
%run gameFunction.ipynb 

In [50]:
#Developer tool - game output simulation
# player configuration -> random function -> game
playerDict = {
    0: {
        'playerName': 'Filip',
        'playerFreq': 70,
        'playerSkill': 5
    },
    1: {
        'playerName': 'Mikołaj',
        'playerFreq': 80,
        'playerSkill': 3
    },
    2: {
        'playerName': 'Matwiej',
        'playerFreq': 50,
        'playerSkill': 3
    },
    3: {
        'playerName': 'Jędras',
        'playerFreq': 70,
        'playerSkill': 4
    },
    4: {
        'playerName': 'Młody',
        'playerFreq': 60,
        'playerSkill': 5
    },
    5: {
        'playerName': 'Szlachetka',
        'playerFreq': 30,
        'playerSkill': 5
    },
}

def checkFrequency(playerDict, playersNumber, maxFactor=3):
    frequencyFactor = random.randint(10,90)
    strictArray = []
    for p in playerDict:
        if playerDict[p].get('playerFreq') >= frequencyFactor: strictArray.append(playerDict[p].get('playerName'))
    if len(strictArray) >= playersNumber: return strictArray
    else:
        additionalDict = {}
        for p in playerDict:
            if playerDict[p].get('playerName') not in strictArray:
                randomFactor = random.randint(1,maxFactor)*playerDict[p].get('playerFreq')
                additionalDict.update({playerDict[p].get('playerName'): randomFactor})
        
        freqList = []
        for p in additionalDict: freqList.append(additionalDict.get(p))
        
        while len(strictArray) < playersNumber:
            for p in additionalDict:
                if len(freqList) > 0:
                    if additionalDict.get(p) == max(freqList): 
                        strictArray.append(p)
                        freqList.pop(freqList.index(max(freqList)))
        
        return strictArray
            
def checkPoints(playerDict, playersArray, randomFactor=True):
#     skille * randomowy mnożnink + ranodm factor
    skillScoreFin = {}
    gamePoints = {}
    
    for p in playerDict:
        if playerDict[p].get('playerName') in playersArray:
            skillScore = playerDict[p].get('playerSkill')*random.randint(1, 4)
            if randomFactor == True: skillScore += random.randint(-5, 5)
            skillScoreFin.update({playerDict[p].get('playerName'):skillScore})
            
    for p in skillScoreFin:
        if skillScoreFin[p] == max(skillScoreFin.values()):
            gamePoints.update({p:{'winner':True, 'skillscore':skillScoreFin[p]}})
        else:
            gamePoints.update({p:{'winner':False, 'skillscore':skillScoreFin[p]}})
            
    return gamePoints

def createGame(playerDict, playerMinNumber=2, gamesMinNumber=1, gamesMaxNumber=5, gamePoints=3):
    #basic game settings
    playersNumber = random.randint(playerMinNumber, len(playerDict))
    gamesNumber = random.randint(gamesMinNumber, gamesMaxNumber)
    playersArray = []
    
    
    #players in game
    possiblePlayerArray = checkFrequency(playerDict, playersNumber)
    
    if len(possiblePlayerArray) == playersNumber: playersArray = possiblePlayerArray
    else:
        while len(playersArray) < playersNumber:
            playerIndex = random.randint(0, len(possiblePlayerArray)-1)
            player = playerDict[playerIndex].get('playerName')
            if player not in playersArray and player in possiblePlayerArray: playersArray.append(player)
    
    
    session = {}
    game = {}
    roundNo = 1
    
    while roundNo < gamesNumber:
        checkSkill = checkPoints(playerDict, playersArray, randomFactor=True)
        game.update(checkSkill)
        session.update({roundNo:game})
        game = {}
        roundNo += 1
    
    session.update({'masterUID':str(uuid.uuid4())})
    return session


# createGame(playerDict)
count = 0
while count < 10:
    print(createGame(playerDict))
    print('---------------------')
    count += 1

{1: {'Filip': {'winner': True, 'skillscore': 17}, 'Mikołaj': {'winner': False, 'skillscore': 9}, 'Jędras': {'winner': False, 'skillscore': 12}, 'Młody': {'winner': False, 'skillscore': 13}}, 2: {'Filip': {'winner': False, 'skillscore': 1}, 'Mikołaj': {'winner': False, 'skillscore': 3}, 'Jędras': {'winner': False, 'skillscore': 13}, 'Młody': {'winner': True, 'skillscore': 24}}, 3: {'Filip': {'winner': True, 'skillscore': 20}, 'Mikołaj': {'winner': False, 'skillscore': 4}, 'Jędras': {'winner': False, 'skillscore': 0}, 'Młody': {'winner': False, 'skillscore': 16}}, 4: {'Filip': {'winner': False, 'skillscore': 12}, 'Mikołaj': {'winner': False, 'skillscore': 5}, 'Jędras': {'winner': False, 'skillscore': 7}, 'Młody': {'winner': True, 'skillscore': 15}}, 'masterUID': '7a86a2de-d41d-4a7a-b612-749cae70a0e1'}
---------------------
{1: {'Filip': {'winner': False, 'skillscore': 17}, 'Mikołaj': {'winner': False, 'skillscore': 11}, 'Matwiej': {'winner': False, 'skillscore': 10}, 'Jędras': {'winner':

In [46]:
#filtr czasowy
def filterByDate(df, startDate=0, startTime='00:00', endDate=0, endTime='23:59'):
    if startDate == 0: startDate = df['timestamp'].values[0]
    if endDate == 0: endDate = df['timestamp'].values[-1]
    df['timestamp'] = pd.to_datetime(df['timestamp'], yearfirst=True)
    # startTime = '00:00'
    # startDate = '2022-02-01'
    # endTime = '23:59'
    # endDate = '2022-02-01'
    df = df.loc[(df['timestamp'] >= f'{startDate} {startTime}') & (df['timestamp'] <= f'{endDate} {endTime}')]
    return df

# osoba, która wygrała najwięcej gier
def overallWinner(df):
    df = df.loc[df['end'] == False]
    dfN = df[['name', 'end']].groupby('name').count().sort_values('end', ascending=False)
    dfDict = dfN.to_dict()
    return dfDict

#osoba, która wygrała najwięcej w danym czasie
def timeWinner(df, startDate=0, startTime='00:00', endDate=0, endTime='23:59'):
    print(startDate)
    dfFilter = filterByDate(df, startDate, startTime, endDate, endTime)
    dfDict = overallWinner(dfFilter)
    return dfDict

#filtr po imieniu
def filterByName(df, playerName):
    dfM = df.loc[df['name'] == playerName]
    dfMap = set(dfM['masterUID'])
    df = df.loc[df['masterUID'].isin(dfMap)]
    dfDict = df.to_dict('records')
    return dfDict
filterByName(df, 'Klaudia')

[{'name': 'Hela',
  'points': 1,
  'word': 'KEN',
  'gameUID': 0,
  'winGames': 0,
  'finGames': 0,
  'end': False,
  'masterUID': '716597bf-bab6-453f-a136-d59a25b263e9',
  'timestamp': '2022-01-31 18:26'},
 {'name': 'Klaudia',
  'points': 3,
  'word': 'KEN',
  'gameUID': 0,
  'winGames': 0,
  'finGames': 0,
  'end': True,
  'masterUID': '716597bf-bab6-453f-a136-d59a25b263e9',
  'timestamp': '2022-01-31 18:26'},
 {'name': 'Hela',
  'points': 2,
  'word': 'KEN',
  'gameUID': 1,
  'winGames': 1,
  'finGames': 1,
  'end': False,
  'masterUID': '716597bf-bab6-453f-a136-d59a25b263e9',
  'timestamp': '2022-01-31 18:26'},
 {'name': 'Klaudia',
  'points': 3,
  'word': 'KEN',
  'gameUID': 1,
  'winGames': 0,
  'finGames': 1,
  'end': True,
  'masterUID': '716597bf-bab6-453f-a136-d59a25b263e9',
  'timestamp': '2022-01-31 18:26'},
 {'name': 'Hela',
  'points': 3,
  'word': 'KEN',
  'gameUID': 2,
  'winGames': 2,
  'finGames': 2,
  'end': True,
  'masterUID': '716597bf-bab6-453f-a136-d59a25b263e9'

{'end': {'Klaudia': 3, 'Hela': 2, 'Paweł': 2, 'Damian': 1, 'Piotr': 1}}

In [13]:
df.loc[df['masterUID'] == '716597bf-bab6-453f-a136-d59a25b263e9']

,name,points,word,gameUID,winGames,finGames,end,masterUID,timestamp
22,Hela,1,KEN,0,0,0,False,716597bf-bab6-453f-a136-d59a25b263e9,2022-01-31 18:26:31.735016
23,Klaudia,3,KEN,0,0,0,True,716597bf-bab6-453f-a136-d59a25b263e9,2022-01-31 18:26:31.735016
24,Hela,2,KEN,1,1,1,False,716597bf-bab6-453f-a136-d59a25b263e9,2022-01-31 18:26:36.109558
25,Klaudia,3,KEN,1,0,1,True,716597bf-bab6-453f-a136-d59a25b263e9,2022-01-31 18:26:36.109558
26,Hela,3,KEN,2,2,2,True,716597bf-bab6-453f-a136-d59a25b263e9,2022-01-31 18:26:42.512791
27,Klaudia,2,KEN,2,0,2,False,716597bf-bab6-453f-a136-d59a25b263e9,2022-01-31 18:26:42.512791


In [11]:
df.value_counts('name')

name
Hela       10
Damian      8
Klaudia     8
Piotr       5
dtype: int64